In [7]:
# Notebook 01 - INGESTÃO DE DADOS
# Pipeline de Big Data - Análise de Churn Telecom
# Equipe: [NOMES DOS MEMBROS]

# ============================================================
# ETAPA 1: CONFIGURAÇÃO DO AMBIENTE
# ============================================================

import pandas as pd
import numpy as np
from datetime import datetime
import os
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

# Criar estrutura de pastas (Arquitetura Medallion)
pastas = ['/content/dados/bronze', '/content/dados/silver', '/content/dados/gold']
for pasta in pastas:
    os.makedirs(pasta, exist_ok=True)

print("✅ Estrutura de pastas criada:")
print("   - /dados/bronze  → Dados brutos")
print("   - /dados/silver  → Dados transformados")
print("   - /dados/gold    → Dados prontos para análise")



✅ Estrutura de pastas criada:
   - /dados/bronze  → Dados brutos
   - /dados/silver  → Dados transformados
   - /dados/gold    → Dados prontos para análise


In [8]:
# ============================================================
# ETAPA 2: INGESTÃO DE DADOS (DATA INGESTION)
# ============================================================

print("\n" + "="*60)
print("🔄 INICIANDO INGESTÃO DE DADOS")
print("="*60)

# Imports adicionais para download
import os
import urllib.request

# Função inteligente para obter dataset
def obter_dataset():
    """
    Obtém o dataset usando múltiplos métodos:
    1. Verifica se já existe localmente
    2. Tenta download automático do GitHub
    3. Solicita upload manual
    """

    # Método 1: Verificar se arquivo já existe
    arquivos_possiveis = [
        'telco_churn.csv',
        'Telco-Customer-Churn.csv',
        'WA_Fn-UseC_-Telco-Customer-Churn.csv'
    ]

    for arquivo in arquivos_possiveis:
        if os.path.exists(arquivo):
            print(f"✅ Arquivo encontrado: {arquivo}")
            return arquivo

    # Método 2: Tentar download automático
    print("📥 Arquivo não encontrado. Tentando download automático...\n")

    try:
        print("⬇️ Baixando dataset do GitHub...")
        url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
        nome_arquivo = 'telco_churn.csv'

        urllib.request.urlretrieve(url, nome_arquivo)

        # Verificar se download funcionou
        if os.path.exists(nome_arquivo) and os.path.getsize(nome_arquivo) > 1000:
            print("✅ Download concluído com sucesso!")
            return nome_arquivo
        else:
            raise Exception("Arquivo baixado está vazio ou corrompido")

    except Exception as e:
        print(f"⚠️ Download automático falhou: {e}\n")

    # Método 3: Upload manual
    print("📤 Por favor, faça upload do arquivo:")
    print("   WA_Fn-UseC_-Telco-Customer-Churn.csv")
    print("   (Clique no botão abaixo)\n")

    uploaded = files.upload()

    if uploaded:
        nome_arquivo = list(uploaded.keys())[0]
        print(f"\n✅ Arquivo recebido: {nome_arquivo}")
        return nome_arquivo
    else:
        raise FileNotFoundError("❌ Nenhum arquivo foi carregado!")

# Executar função para obter dataset
nome_arquivo = obter_dataset()

# Ler dados brutos
print(f"\n📖 Lendo arquivo: {nome_arquivo}")
df_bronze = pd.read_csv(nome_arquivo)

# Adicionar metadados de ingestão
df_bronze['data_ingestao'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_bronze['fonte'] = 'Kaggle_TelcoChurn'

# Salvar na camada Bronze (dados brutos)
caminho_bronze = '/content/dados/bronze/telco_churn_raw.csv'
df_bronze.to_csv(caminho_bronze, index=False)

print(f"\n✅ Dados ingeridos com sucesso!")
print(f"   📊 Total de registros: {len(df_bronze):,}")
print(f"   📁 Arquivo salvo em: {caminho_bronze}")



🔄 INICIANDO INGESTÃO DE DADOS
✅ Arquivo encontrado: telco_churn.csv

📖 Lendo arquivo: telco_churn.csv

✅ Dados ingeridos com sucesso!
   📊 Total de registros: 7,043
   📁 Arquivo salvo em: /content/dados/bronze/telco_churn_raw.csv


In [9]:


# ============================================================
# ETAPA 3: EXPLORAÇÃO INICIAL DOS DADOS
# ============================================================

print("\n" + "="*60)
print("🔍 ANÁLISE EXPLORATÓRIA INICIAL")
print("="*60)

print(f"\n📐 Dimensões do dataset: {df_bronze.shape}")
print(f"   - Linhas: {df_bronze.shape[0]:,}")
print(f"   - Colunas: {df_bronze.shape[1]}")

print("\n📋 Primeiras 5 linhas:")
print(df_bronze.head())

print("\n📊 Informações dos tipos de dados:")
print(df_bronze.info())

print("\n🔢 Estatísticas descritivas:")
print(df_bronze.describe())

print("\n❓ Valores ausentes por coluna:")
missing = df_bronze.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("   ✅ Nenhum valor ausente encontrado!")

print("\n📈 Distribuição da variável alvo (Churn):")
print(df_bronze['Churn'].value_counts())
print(f"\n   Taxa de Churn: {(df_bronze['Churn']=='Yes').mean()*100:.2f}%")




🔍 ANÁLISE EXPLORATÓRIA INICIAL

📐 Dimensões do dataset: (7043, 23)
   - Linhas: 7,043
   - Colunas: 23

📋 Primeiras 5 linhas:
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... StreamingTV  \
0  No phone service             DSL             No  ...          No   
1                No             DSL            Yes  ...          No   
2                No             DSL            Yes  ...          No   
3  No phone service             DSL            Yes  ...          No   
4                No     Fiber optic  

In [10]:

# ============================================================
# ETAPA 4: VALIDAÇÕES DE QUALIDADE
# ============================================================

print("\n" + "="*60)
print("✔️ VALIDAÇÕES DE QUALIDADE DOS DADOS")
print("="*60)

# Verificar duplicatas
duplicatas = df_bronze.duplicated(subset='customerID').sum()
print(f"\n🔍 Duplicatas encontradas: {duplicatas}")

# Verificar consistência de valores
print("\n📊 Valores únicos em colunas categóricas:")
colunas_categoricas = ['gender', 'Partner', 'Dependents', 'PhoneService',
                       'InternetService', 'Contract', 'PaperlessBilling', 'Churn']
for col in colunas_categoricas:
    if col in df_bronze.columns:
        print(f"   - {col}: {df_bronze[col].unique()}")

# Verificar range de valores numéricos
print("\n📏 Range de valores numéricos:")
print(f"   - tenure: {df_bronze['tenure'].min()} a {df_bronze['tenure'].max()} meses")
print(f"   - MonthlyCharges: R$ {df_bronze['MonthlyCharges'].min():.2f} a R$ {df_bronze['MonthlyCharges'].max():.2f}")

print("\n" + "="*60)
print("✅ INGESTÃO COMPLETA - Dados prontos para transformação!")
print("="*60)

# Salvar relatório de ingestão
relatorio = f"""
RELATÓRIO DE INGESTÃO DE DADOS
Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
==========================================
Fonte: {nome_arquivo}
Registros ingeridos: {len(df_bronze):,}
Colunas: {df_bronze.shape[1]}
Duplicatas: {duplicatas}
Taxa de Churn: {(df_bronze['Churn']=='Yes').mean()*100:.2f}%
==========================================
"""

with open('/content/dados/bronze/relatorio_ingestao.txt', 'w') as f:
    f.write(relatorio)

print("\n📄 Relatório de ingestão salvo em: /content/dados/bronze/relatorio_ingestao.txt")


✔️ VALIDAÇÕES DE QUALIDADE DOS DADOS

🔍 Duplicatas encontradas: 0

📊 Valores únicos em colunas categóricas:
   - gender: ['Female' 'Male']
   - Partner: ['Yes' 'No']
   - Dependents: ['No' 'Yes']
   - PhoneService: ['No' 'Yes']
   - InternetService: ['DSL' 'Fiber optic' 'No']
   - Contract: ['Month-to-month' 'One year' 'Two year']
   - PaperlessBilling: ['Yes' 'No']
   - Churn: ['No' 'Yes']

📏 Range de valores numéricos:
   - tenure: 0 a 72 meses
   - MonthlyCharges: R$ 18.25 a R$ 118.75

✅ INGESTÃO COMPLETA - Dados prontos para transformação!

📄 Relatório de ingestão salvo em: /content/dados/bronze/relatorio_ingestao.txt


In [11]:
# Adicionar no final do notebook:
from google.colab import files
files.download('/content/dados/bronze/telco_churn_raw.csv')
files.download('/content/dados/bronze/relatorio_ingestao.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>